In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
# K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block 
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [3]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block 
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a',padding='valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(F2, (f, f), strides = (1,1), name = conv_name_base + '2b',padding='same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1, 1), strides = (1,1), name = conv_name_base + '2c',padding='valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1',padding='valid', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [4]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Implementation of the popular ResNet50 with modifications the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> AVGPOOL -> TOPLAYER
    
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(32, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    # Stage 2
    X = convolutional_block(X, f = 3, filters = [32, 32, 128], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [32, 32, 128], stage=2, block='b')
    X = identity_block(X, 3, [32, 32, 128], stage=2, block='c')
    # Stage 3
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='c')
    X = identity_block(X, 3, [64, 64, 256], stage=3, block='d')


    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D()(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [5]:
model = ResNet50(input_shape = (48, 48, 1), classes = 7)

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).

In [8]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 54, 54, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 24, 24, 32)   1600        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 24, 24, 32)   128         conv1[0][0]                      
___________________________________________________________________________________________

In [9]:
import pandas as pd

In [10]:
dataset = pd.read_csv('28577_36420_bundle_archive/fer2013.csv')

In [11]:
training_data = dataset[dataset['Usage']=='Training']

In [12]:
dev_set = dataset[dataset['Usage']=='PublicTest']

In [13]:
test_set = dataset[dataset['Usage']=='PrivateTest']

In [14]:
dev_set

,emotion,pixels,Usage
28709,0,254 254 254 254 254 249 255 160 2 58 53 70 77 ...,PublicTest
28710,1,156 184 198 202 204 207 210 212 213 214 215 21...,PublicTest
28711,4,69 118 61 60 96 121 103 87 103 88 70 90 115 12...,PublicTest
28712,6,205 203 236 157 83 158 120 116 94 86 155 180 2...,PublicTest
28713,3,87 79 74 66 74 96 77 80 80 84 83 89 102 91 84 ...,PublicTest
...,...,...,...
32293,4,178 176 172 173 173 174 176 173 166 166 206 22...,PublicTest
32294,3,25 34 42 44 42 47 57 59 59 58 54 51 50 56 63 6...,PublicTest
32295,4,255 255 255 255 255 255 255 255 255 255 255 25...,PublicTest
32296,4,33 25 31 36 36 42 69 103 132 163 175 183 187 1...,PublicTest


In [15]:
X_train,Y_Train = training_data['pixels'],training_data['emotion']

In [16]:
X_train2 = np.zeros((X_train.shape[0],48*48)) 
for i in range(X_train2.shape[0]):
    p = X_train[i].split(' ')
    for j in range(X_train2.shape[1]):
        X_train2[i][j] = int(p[j])

In [17]:
Y_train2 = np.array(Y_Train)

In [18]:
X_train3 = X_train2.reshape(X_train2.shape[0],48,48)

In [19]:
Y_train = Y_train2
X_train = X_train3

In [20]:
dev_set = dev_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

In [21]:
X_dev,Y_dev = dev_set['pixels'],dev_set['emotion']
X_dev2 = np.zeros((X_dev.shape[0],48*48)) 
for i in range(X_dev2.shape[0]):
    p = X_dev[i].split(' ')
    for j in range(X_dev2.shape[1]):
        X_dev2[i][j] = int(p[j])
Y_dev2 = np.array(Y_dev)
X_dev3 = X_dev2.reshape(X_dev2.shape[0],48,48)
Y_dev = Y_dev2
X_dev = X_dev3

In [22]:
X_test,Y_test = test_set['pixels'],test_set['emotion']
X_test2 = np.zeros((X_test.shape[0],48*48)) 
for i in range(X_test2.shape[0]):
    p = X_test[i].split(' ')
    for j in range(X_test2.shape[1]):
        X_test2[i][j] = int(p[j])
Y_test2 = np.array(Y_test)
X_test3 = X_test2.reshape(X_test2.shape[0],48,48)
Y_test = Y_test2
X_test = X_test3

In [23]:
# # Preprocess test train dev set

# Normalize image vectors
X_train = X_train/255.0
X_dev = X_dev/255.0
X_test = X_test/255.0

# # Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train, 7).T
Y_dev = convert_to_one_hot(Y_dev, 7).T
Y_test = convert_to_one_hot(Y_test, 7).T

X_train = X_train.reshape(X_train.shape[0],48,48,1)
X_dev = X_dev.reshape(X_dev.shape[0],48,48,1)
X_test = X_test.reshape(X_test.shape[0],48,48,1)
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 28709
number of test examples = 3589
X_train shape: (28709, 48, 48, 1)
Y_train shape: (28709, 7)
X_test shape: (3589, 48, 48, 1)
Y_test shape: (3589, 7)


In [24]:
validation_data = (X_dev,Y_dev)

In [25]:
datagen = image.ImageDataGenerator(featurewise_center=True,
                                  featurewise_std_normalization=True,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True)
datagen.fit(X_train)

In [ ]:
model.fit(datagen.flow(X_train, Y_train,batch_size=32), epochs = 50,steps_per_epoch=len(X_train)/32,validation_data=validation_data)

Epoch 1/50
898/897 [==============================] - 53s 59ms/step - loss: 2.0304 - accuracy: 0.2441 - val_loss: 1.8578 - val_accuracy: 0.2293
Epoch 2/50
898/897 [==============================] - 36s 40ms/step - loss: 1.7386 - accuracy: 0.3144 - val_loss: 1.8963 - val_accuracy: 0.2017
Epoch 3/50
898/897 [==============================] - 36s 40ms/step - loss: 1.6041 - accuracy: 0.3726 - val_loss: 1.8880 - val_accuracy: 0.2733
Epoch 4/50
898/897 [==============================] - 36s 40ms/step - loss: 1.5021 - accuracy: 0.4191 - val_loss: 1.8023 - val_accuracy: 0.2940
Epoch 5/50
898/897 [==============================] - 36s 41ms/step - loss: 1.4257 - accuracy: 0.4518 - val_loss: 1.7414 - val_accuracy: 0.3254
Epoch 6/50
898/897 [==============================] - 37s 41ms/step - loss: 1.3658 - accuracy: 0.4759 - val_loss: 1.7681 - val_accuracy: 0.3004
Epoch 7/50
898/897 [==============================] - 40s 44ms/step - loss: 1.3310 - accuracy: 0.4939 - val_loss: 1.6210 - val_accuracy:

In [ ]:
preds = model.evaluate(X_dev, Y_dev)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [9]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

3589/3589 [==============================] - 3s 767us/step
Loss = 1.033392020414785
Test Accuracy = 0.6288659572601318


In [46]:
model.save('CustomResnet.h5')